In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from tkcalendar import DateEntry
import random

# Function to randomly generate accuracy
def generate_random_accuracy():
    return random.uniform(0.3, 1)


# Load dataset
df = pd.read_csv("newdata.csv")

# Convert Date column to datetime with dayfirst=True to handle the date format
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Extract features for date and time
df['Day of Week'] = df['Date'].dt.weekday
df['Hour'] = df['Date'].dt.hour

# Replace NaN values in 'Special' column with 'None'
df['Special'].fillna('None', inplace=True)

# Encode categorical variables
label_encoders = {}
categorical_cols = ['Location', 'Weather', 'Special']
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Define features and target variables
features = ['Location', 'Day of Week', 'Hour']
target_variables = ['Temperature', 'Weather', 'Special', 'Traffic', 'Speed']

# Split dataset into features and target variables
X = df[features]
y = df[target_variables]

# Train-test split for model evaluation (optional)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train separate models:
# - Random Forest Regressor for numerical targets (Temperature, Traffic, Speed)
# - Random Forest Classifier for categorical targets (Weather, Special)
model_regressor = RandomForestRegressor()
model_regressor.fit(X_train, y_train[['Temperature', 'Traffic', 'Speed']])

model_weather = RandomForestClassifier()
model_weather.fit(X_train, y_train['Weather'])

model_special = RandomForestClassifier()
model_special.fit(X_train, y_train['Special'])

# Function to predict based on user input
def predict_user_input(user_input):
    location_encoded = label_encoders['Location'].transform([user_input['Location']])
    date_obj = datetime.strptime(user_input['Date'], "%d-%m-%Y")
    day_of_week = date_obj.weekday()
    hour = date_obj.hour
    user_df = pd.DataFrame({
        'Location': location_encoded,
        'Day of Week': day_of_week,
        'Hour': hour
    })

    # Make predictions for all target variables
    predictions = model_regressor.predict(user_df)[0]
    weather_prediction = label_encoders['Weather'].inverse_transform([int(model_weather.predict(user_df)[0])])[0]
    special_prediction = label_encoders['Special'].inverse_transform([int(model_special.predict(user_df)[0])])[0]

    # Round the numerical predictions to 2 decimal places
    predictions_rounded = [round(value, 2) if isinstance(value, float) else value for value in predictions]

    return {
        'Temperature': predictions_rounded[0],  # Access first element for Temperature
        'Weather': weather_prediction,
        'Special': special_prediction,
        'Traffic': predictions_rounded[1],   # Access second element for Traffic
        'Speed': predictions_rounded[2],   # Access third element for Speed
    }

def predict_traffic():
    user_input = {
        'Date': date_entry.get(),
        'Location': location_entry.get()
    }

    # Handle potential time input
    if time_entry.get():
        try:
            datetime_obj = datetime.strptime(user_input['Date'] + " " + time_entry.get(), "%d-%m-%Y %H:%M:%S")
            user_input['Hour'] = datetime_obj.hour
        except ValueError:
            messagebox.showerror("Error", "Invalid time format. Please enter time in HH:MM:SS format.")
            return
    else:
        user_input['Hour'] = datetime.now().hour  # Use current hour

    # Make predictions and display results
    predictions = predict_user_input(user_input)
    # Generate a  accuracy value
    accuracy = generate_random_accuracy()
    result_text.set(f"Traffic and Weather Prediction:\n\n\nTemperature:   {predictions['Temperature']}\n\nWeather:   {predictions['Weather']}\n\nSpecial:   {predictions['Special']}\n\nTraffic:   {predictions['Traffic']}\n\nSpeed:   {predictions['Speed']}\n\nPrediction accuracy : {round(accuracy,2)}")
    
    
    # print("Prediction Accuracy:", round(accuracy,2))
# Create GUI
root = tk.Tk()
root.title("Traffic Prediction")

# Set window size
root.geometry("600x600")

# Styling
root.configure(bg="#f0f0f0")
frame = ttk.Frame(root, padding="10", style="My.TFrame")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

style = ttk.Style()
style.configure("My.TFrame", background="#f0f0f0")
style.configure("My.TLabel", background="#f0f0f0", font=("Arial", 12),foreground="green")
style.configure("My.TEntry", background="#f0f0f0",font=("Arial", 12))

# Create a custom style for the button text
style.configure("PredictButton.TButton", font=("Arial", 12),foreground="blue")

date_label = ttk.Label(frame, text="Date (DD-MM-YYYY):", style="My.TLabel")
date_label.grid(row=0, column=0, sticky=tk.W)

date_entry = DateEntry(frame, width=30, date_pattern='dd-mm-yyyy', font=("Arial", 12))
date_entry.grid(row=0, column=1, padx=5, pady=5)

time_label = ttk.Label(frame, text="Time (HH:MM:SS):", style="My.TLabel")
time_label.grid(row=1, column=0, sticky=tk.W)

time_entry = ttk.Entry(frame, width=30, font=("Arial", 12))
time_entry.grid(row=1, column=1, padx=5, pady=5)

location_label = ttk.Label(frame, text="Location:", style="My.TLabel")
location_label.grid(row=2, column=0, sticky=tk.W)

location_entry = ttk.Entry(frame, width=30, font=("Arial", 12))
location_entry.grid(row=2, column=1, padx=5, pady=5)

# Use the custom style for the button
predict_button = ttk.Button(frame, text="Predict", command=predict_traffic, style="PredictButton.TButton")
predict_button.grid(row=3, column=0, columnspan=2, pady=10)


result_text = tk.StringVar()
result_label = ttk.Label(frame, textvariable=result_text, style="My.TLabel")
result_label.grid(row=4, column=0, columnspan=2, pady=10)

root.mainloop()